# MLOps Final Project:

Comparing performance of fine-tuned LLM vs out of the box pretrained LLM on sentiment analysis task for IMDB reviews dataset

# Data Preprocessing

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
import loralib as lora
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# Load the dataset
df = pd.read_csv('movie_short.csv')

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True)

# Convert to torch dataset
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_df['label'].tolist())
val_dataset = IMDbDataset(val_encodings, val_df['label'].tolist())

# Define the evaluation function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
    acc = accuracy_score(labels, pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Load the pre-trained DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Evaluate the pre-trained model
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=0,  # No training, just evaluation
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,
    do_eval=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

eval_result = trainer.evaluate()

# Fine-tune the DistilBERT model without LoRA
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

eval_result_fine_tuned = trainer.evaluate()


lora_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules= ["attention.q_lin", "attention.k_lin", "attention.v_lin"], #["query", "key", "value"],  # Specify the target modules
    bias="none",
)

lora_model = get_peft_model(lora_model, lora_config)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

eval_result_lora = trainer.evaluate()


# Print the evaluation results
print("Pre-trained DistilBERT evaluation:")
print(eval_result)

print("Regular fine-tuned DistilBERT evaluation:")
print(eval_result_fine_tuned)

print("LoRA fine-tuned DistilBERT evaluation:")
print(eval_result_lora)


/opt/homebrew/anaconda3/envs/DistributedComputing/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

{'loss': 0.6845, 'grad_norm': 1.402540922164917, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}
{'loss': 0.6878, 'grad_norm': 1.3427975177764893, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.4}
{'loss': 0.6826, 'grad_norm': 1.4069232940673828, 'learning_rate': 3e-06, 'epoch': 0.6}
{'loss': 0.6958, 'grad_norm': 1.7702349424362183, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.8}
{'loss': 0.6714, 'grad_norm': 1.014061689376831, 'learning_rate': 5e-06, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

...

{'loss': 0.67, 'grad_norm': 1.5083074569702148, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}
{'loss': 0.6747, 'grad_norm': 1.4883174896240234, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.4}
{'loss': 0.6623, 'grad_norm': 1.5165575742721558, 'learning_rate': 3e-06, 'epoch': 0.6}
{'loss': 0.6727, 'grad_norm': 1.9634284973144531, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.8}
{'loss': 0.6407, 'grad_norm': 1.265491008758545, 'learning_rate': 5e-06, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.648670494556427, 'eval_accuracy': 0.805, 'eval_f1': 0.7936507936507936, 'eval_precision': 0.8241758241758241, 'eval_recall': 0.7653061224489796, 'eval_runtime': 4.4635, 'eval_samples_per_second': 44.808, 'eval_steps_per_second': 2.913, 'epoch': 1.0}
{'train_runtime': 61.1212, 'train_samples_per_second': 13.072, 'train_steps_per_second': 0.818, 'train_loss': 0.6640888595581055, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

Pre-trained DistilBERT evaluation:
{'eval_loss': 0.690392017364502, 'eval_accuracy': 0.52, 'eval_f1': 0.04, 'eval_precision': 1.0, 'eval_recall': 0.02040816326530612, 'eval_runtime': 4.4976, 'eval_samples_per_second': 44.468, 'eval_steps_per_second': 2.89}
Regular fine-tuned DistilBERT evaluation:
{'eval_loss': 0.6795913577079773, 'eval_accuracy': 0.655, 'eval_f1': 0.46511627906976744, 'eval_precision': 0.967741935483871, 'eval_recall': 0.30612244897959184, 'eval_runtime': 4.3478, 'eval_samples_per_second': 46.0, 'eval_steps_per_second': 2.99, 'epoch': 1.0}
LoRA fine-tuned DistilBERT evaluation:
{'eval_loss': 0.648670494556427, 'eval_accuracy': 0.805, 'eval_f1': 0.7936507936507936, 'eval_precision': 0.8241758241758241, 'eval_recall': 0.7653061224489796, 'eval_runtime': 4.3584, 'eval_samples_per_second': 45.889, 'eval_steps_per_second': 2.983, 'epoch': 1.0}
